In [38]:
'''
Created on Nov 18, 2017

Process refSeq gtf file.

The refseq table schema is copied below:

 	Database: hg19    Primary Table: refGene    Row Count: 69,703   Data last updated: 2017-11-13
Format description: A gene prediction with some additional info.
field	example	SQL type	info	description
bin	585	smallint(5) unsigned	range	Indexing field to speed chromosome range queries.
name	NR_148357	varchar(255)	values	Name of gene (usually transcript_id from GTF)
chrom	chr1	varchar(255)	values	Reference sequence chromosome or scaffold
strand	+	char(1)	values	+ or - for strand
txStart	11868	int(10) unsigned	range	Transcription start position (or end position for minus strand item)
txEnd	14362	int(10) unsigned	range	Transcription end position (or start position for minus strand item)
cdsStart	14362	int(10) unsigned	range	Coding region start (or end position for minus strand item)
cdsEnd	14362	int(10) unsigned	range	Coding region end (or start position for minus strand item)
exonCount	3	int(10) unsigned	range	Number of exons
exonStarts	11868,12612,13220,	longblob	 	Exon start positions (or end positions for minus strand item)
exonEnds	12227,12721,14362,	longblob	 	Exon end positions (or start positions for minus strand item)
score	0	int(11)	range	score
name2	LOC102725121	varchar(255)	values	Alternate name (e.g. gene_id from GTF)
cdsStartStat	unk	enum('none', 'unk', 'incmpl', 'cmpl')	values	enum('none','unk','incmpl','cmpl')
cdsEndStat	unk	enum('none', 'unk', 'incmpl', 'cmpl')	values	enum('none','unk','incmpl','cmpl')
exonFrames	-1,-1,-1,	longblob	 	Exon frame {0,1,2}, or -1 if no frame for exon

Some additional info about refseq gtf is here: https://www.ncbi.nlm.nih.gov/refseq/about/

Note: There are a total of 27957 unique gene_symbols (i.e. "name2" below) in this refseq df. 
'''

import os
import numpy as np
import pandas as pd

gtf_refseq = "hg19.refSeqGenes.gtf"

In [13]:
df = pd.read_csv(gtf_refseq, sep="\t", header=0)
df.head(3)

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,0,NM_001350218,chr1,+,66999638,67216822,67000041,67208778,23,"66999638,67091529,67098752,67105459,67108492,6...","67000051,67091593,67098777,67105516,67108547,6...",0,SGIP1,cmpl,cmpl,"0,1,2,0,0,1,0,0,0,1,2,1,1,1,0,1,1,2,2,0,2,1,1,"
1,0,NM_001350217,chr1,+,66999638,67216822,67000041,67208778,25,"66999638,67091529,67098752,67099762,67105459,6...","67000051,67091593,67098777,67099846,67105516,6...",0,SGIP1,cmpl,cmpl,"0,1,2,0,0,0,1,0,0,0,1,2,1,1,1,1,0,1,1,2,2,0,2,..."
2,0,NM_001308203,chr1,+,66999251,67216822,67000041,67208778,22,"66999251,66999928,67091529,67098752,67105459,6...","66999355,67000051,67091593,67098777,67105516,6...",0,SGIP1,cmpl,cmpl,"-1,0,1,2,0,0,1,0,1,2,1,1,1,0,1,1,2,2,0,2,1,1,"


In [14]:
'''Get the tss, and tss +/- 500bp fields'''
df["tss"] = [x[0] if x[2] == "+" else x[1] for x in zip(df["txStart"], df["txEnd"], df["strand"])]

df["tss_m500bp"] = df["tss"] - 500
df["tss_p500bp"] = df["tss"] + 500

In [24]:
df_tidied = df[["chrom", "tss_m500bp", "tss_p500bp", "name2"]]  # same "name2" can have different "name" fields, so not including the latter
print(df_tidied.shape)

(69703, 4)


In [35]:
'''Remove duplicates'''
df_tidied = df_tidied.sort_values(by=["name2"])
df_tidied = df_tidied.drop_duplicates()
print("Note that close to half rows are removed by removing the duplicates. New shape is", df_tidied.shape)

('Note that close to half rows are removed by removing the duplicates. New shape is', (38923, 4))


In [49]:
'''For the genes that are still not uniquified - and there are roughly 10k of them left - drop duplicates randomly.
Idea from: https://stackoverflow.com/questions/22864878/dropping-duplicates-randomly'''
idx = np.random.permutation(np.arange(len(df_tidied)))  
df_tidied = df_tidied.iloc[idx].drop_duplicates(subset="name2")
'''Now sort by pos'''
df_tidied = df_tidied.sort_values(by=["chrom", "tss_m500bp"])
print(df_tidied.shape)

In [52]:
df_tidied.head()

,chrom,tss_m500bp,tss_p500bp,name2
1926,chr1,11373,12373,DDX11L1
1929,chr1,16936,17936,MIR6859-2
1927,chr1,28870,29870,WASH7P
1933,chr1,29865,30865,MIR1302-9
1932,chr1,29865,30865,MIR1302-2


In [54]:
df_tidied.to_csv("hg19.refSeqTSSs.pm500bp.txt", sep="\t", header=None, index=None)